In [ ]:
import numpy as np
from sklearn.svm import SVC
from tensorflow.keras import layers, models

# Load the original CNN models for MFCCs and phone posterior probs
original_model1 = tf.keras.models.load_model('/content/drive/MyDrive/saved models/CNN_mfcc.hdf5')
original_model2 = tf.keras.models.load_model('/content/drive/MyDrive/saved models/CNN_pos.hdf5')

# Define the shape of the input data
input_shape_mfcc = (number_of_frames, 13, 1)
input_shape_pos = (number_of_frames, 32, 1)

# Create the first input branch for MFCCs
input1 = layers.Input(shape=input_shape_mfcc)
frozen_layers1 = original_model1.layers[:7]  # Freeze the first 7 layers of the original model

# Create a new model with frozen layers for MFCCs
frozen_model1 = tf.keras.Sequential(frozen_layers1)
for layer in frozen_model1.layers:
    layer.trainable = False

x1 = frozen_model1(input1)  # Pass input1 through the frozen layers

# Create the second input branch for phone posterior probs
input2 = layers.Input(shape=input_shape_pos)
frozen_layers2 = original_model2.layers[:7]  # Freeze the first 7 layers of the original model

# Create a new model with frozen layers for phone posterior probs
frozen_model2 = tf.keras.Sequential(frozen_layers2)
for layer in frozen_model2.layers:
    layer.trainable = False

x2 = frozen_model2(input2)  # Pass input2 through the frozen layers

# Merge the branches
merged = layers.concatenate([x1, x2])

# Add more layers if needed
merged = layers.Dropout(0.5)(merged)  # Dropout layer for regularization
merged = layers.Dense(128, activation='relu')(merged)  # Fully connected layer with ReLU activation
merged = layers.Dropout(0.3)(merged)  # Dropout layer for regularization

# Add the final classification layer
output = layers.Dense(1, activation='sigmoid')(merged)  # Output layer with sigmoid activation

# Create the model
model = models.Model(inputs=[input1, input2], outputs=output)  # Define the input and output of the model

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()  # Display model architecture summary
